# osdm demo

In [89]:
import os
import requests
import json
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

True

## Get Secrets from .env

In [90]:
KEY_SECRET = os.getenv('KEY_SECRET')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')

## Master Data

### Agent Token

In [91]:
url = "{{api-host}}/oauth/v2/token".replace("{{api-host}}", "https://api.osdm-demo-test.cloud.sqills.com")

payload = {
  "grant_type": "https://com.sqills.s3.oauth.agent",
  "username": "{{agent.username}}",
  "password": "{{agent.password}}"
}

payload['username'] = payload['username'].replace("{{agent.username}}", USERNAME)
payload['password'] = payload['password'].replace("{{agent.password}}", PASSWORD)

headers = {
  'Authorization': 'Basic {{auth.key_secret}}',
  'Content-Type': 'application/json'
}

headers['Authorization'] = headers['Authorization'].replace("{{auth.key_secret}}", KEY_SECRET)

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

token = json.loads(response.text)['access_token']

### Products

In [92]:
url = "https://osdm-demo-test.osdm-adapter.s3p.cloud/v2/products"

payload = ""

headers = {
  'Authorization': 'Bearer {{agent.token}}'
}

headers['Authorization'] = headers['Authorization'].replace("{{agent.token}}", token)

response = requests.request("GET", url, headers=headers, data=json.dumps(payload))

df = pd.json_normalize(json.loads(response.text)['products'])
df.head()

,code,id,owner,description,summary
0,BIKE_01,BIKE_01,osdm-demo-test,NaN,NaN
1,CAN_FEE1,CAN_FEE1,osdm-demo-test,Cancellation fee - Flexible products,Cancellation fee - Flexible products
2,CAN_FEE2,CAN_FEE2,osdm-demo-test,Cancellation fee - Medium Flex products,Cancellation fee - Medium Flex products
3,CAN_FEE3,CAN_FEE3,osdm-demo-test,Cancellation fee - Low fixed products,Cancellation fee - Low fixed products
4,LUGGAGE,LUGGAGE,osdm-demo-test,Standard luggage Sqills OSDM sandbox,Standard luggage Sqills OSDM sandbox


### Places

In [93]:
url = "https://osdm-demo-test.osdm-adapter.s3p.cloud/v2/places"

payload = {
  "placeInput": {
    "name": "hb"
  },
  "numberOfResults": 10
}
    
headers = {
  'Authorization': 'Bearer {{agent.token}}',
  'Content-Type': 'application/json'
}

headers['Authorization'] = headers['Authorization'].replace("{{agent.token}}", token)

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

df = pd.json_normalize(json.loads(response.text)['places'])
df

,id,name,address.code,stopPlace.name,stopPlace.ref.objectType,stopPlace.ref.name,stopPlace.ref.stopPlaceRef
0,urn:uic:stn:8015458,Köln Hbf,DEKOH,Köln Hbf,StopPlaceRef,Köln Hbf,urn:uic:stn:8015458
1,urn:uic:stn:8011068,Frankfurt (Main) Hbf,DEFRA,Frankfurt (Main) Hbf,StopPlaceRef,Frankfurt (Main) Hbf,urn:uic:stn:8011068
2,urn:uic:stn:8503000,Zürich HB,CHAJD,Zürich HB,StopPlaceRef,Zürich HB,urn:uic:stn:8503000


### Trips

In [94]:
url = "https://osdm-demo-test.osdm-adapter.s3p.cloud/v2/trips-collection"

payload = {
  "departureTime": "2023-05-15T18:25:43.511Z",
  "origin": {
    "codeList": "UIC",
    "code": "8400058",
    "stopPlaceRef": "urn:uic:stn:8400058"
  },
  "destination": {
    "codeList": "UIC",
    "code": "8727100",
    "stopPlaceRef": "urn:uic:stn:8727100"
  }
}

headers = {
  'Authorization': 'Bearer {{agent.token}}',
  'Content-Type': 'application/json'
}

headers['Authorization'] = headers['Authorization'].replace("{{agent.token}}", token)

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

trip_id = json.loads(response.text)['id']

trip_id

'hTqkqsbZWer46f4N9BuKah_3cXcwnAu35li7G11Ik3w1M25c-vdS_n4U4g3wisyGI1E73w-tq4ioAqwvfDEwRKVFDBzBoA6vMoOxvXRaGtuCvDfXqRDbmiBbhU0BorSWSqS5tzo6yFtCN79iws5jU5bbny3ge7nUDZNc'

### Trip

In [95]:
url = "https://osdm-demo-test.osdm-adapter.s3p.cloud/v2/trips/" + trip_id

payload = ""

headers = {
  'Authorization': 'Bearer {{agent.token}}',
  'Content-Type': 'application/json'
}

headers['Authorization'] = headers['Authorization'].replace("{{agent.token}}", token)

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

response

<Response [405]>

### Offers

In [96]:
url = "https://osdm-demo-test.osdm-adapter.s3p.cloud/v2/offers"

payload = {
  "objectType": "string",
  "tripSpecifications": [
    {
      "externalRef": "string",
      "legs": [
        {
          "externalRef": "string",
          "timedLeg": {
            "start": {
              "stopPlaceRef": {
                "stopPlaceRef": "urn:uic:stn:8400058"
              },
              "serviceDeparture": {
                "timetabledTime": "2023-05-15T18:25:43.511Z"
              }
            },
            "end": {
              "stopPlaceRef": {
                "stopPlaceRef": "urn:uic:stn:8727100"
              },
              "serviceArrival": {
                "timetabledTime": "2023-05-15T18:25:43.511Z"
              }
            },
            "service": {
              "publishedServiceName": "",
              "vehicleNumbers": [
                ""
              ],
              "carriers": [
                {
                  "ref": "",
                  "name": ""
                }
              ]
            }
          }
        }
      ],
      "isPartOfInternationalTrip": False
    }
  ],
  "offerSearchCriteria": {
    "currency": "EUR"
  },
  "anonymousPassengerSpecifications": [
    {
      "externalRef": "",
      "type": "PERSON"
    }
  ],
  "embed": [
    "ALL"
  ]
}

headers = {
  'Authorization': 'Bearer {{agent.token}}',
  'Content-Type': 'application/json'
}

headers['Authorization'] = headers['Authorization'].replace("{{agent.token}}", token)

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

In [97]:
response.text

'{"title":"Bad Request","status":400,"detail":"non_conform_request {description: Request does not conform to specification, v2_error_body: [\\"field body\\" [\\"field travels\\" [[\\"at index 0\\" [\\"field routes\\" [[\\"at index 0\\" [[\\"field legs\\" [[\\"at index 0\\" [[\\"field service_name\\" [\\"expected service name\\" [\\"Name must not be empty\\" \\"\\"]]]]]]]]]]]]]]]}"}'

## Sales Flow

### 2. Navigate

In [98]:
url = "https://api.osdm-demo-test.cloud.sqills.com/api/v2/orientation/navigate"

payload = {
  "travels": [
    {
      "origin": "8015458",
      "destination": "8503000",
      "departure": "2023-05-15",
      "max_transfers": 0
    }
  ]
}

headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'Accept-Language': '{{locale}}',
  'Authorization': 'Bearer {{agent.token}}'
}

headers['Authorization'] = headers['Authorization'].replace("{{agent.token}}", token)

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

data = json.loads(response.text)

data

{'data': {'travels': [{'routes': [], 'id': 'travel_1'}]}}

## Booking a Ticket

### 1. Lookup Location Code

In [99]:
url = "https://osdm-demo-test.osdm-adapter.s3p.cloud/v2/places"

payload = {
  "placeInput": {
    "name": "Basel"
  },
  "numberOfResults": 10
}
    
headers = {
  'Authorization': 'Bearer {{agent.token}}',
  'Content-Type': 'application/json'
}

headers['Authorization'] = headers['Authorization'].replace("{{agent.token}}", token)

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

json.loads(response.text)

{'places': [{'address': {'code': 'CHBAS'},
   'id': 'urn:uic:stn:8500010',
   'name': 'Basel SBB',
   'stopPlace': {'name': 'Basel SBB',
    'ref': {'objectType': 'StopPlaceRef',
     'name': 'Basel SBB',
     'stopPlaceRef': 'urn:uic:stn:8500010'}}}]}

### 2. Request Offers

In [105]:
url = "https://osdm-demo-test.osdm-adapter.s3p.cloud/v2/offers"

payload = {
  "objectType": "string",
  "tripSpecifications": [
    {
      "externalRef": "string",
      "legs": [
        {
          "externalRef": "string",
          "timedLeg": {
            "start": {
              "stopPlaceRef": {
                "stopPlaceRef": "urn:uic:stn:8500010"
              },
              "serviceDeparture": {
                "timetabledTime": "2022-05-15T00:00:00+00:00"
              }
            },
            "end": {
              "stopPlaceRef": {
                "stopPlaceRef": "urn:uic:stn:8503000"
              },
              "serviceArrival": {
                "timetabledTime": "2022-05-16T00:00:00+00:00"
              }
            },
            "service": {
              "publishedServiceName": "OSDM_IC",
              "vehicleNumbers": [
                "OSDM_IC"
              ],
              "carriers": [
                {
                  "ref": "OSDM_IC",
                  "name": "OSDM_IC"
                }
              ]
            }
          }
        }
      ],
      "isPartOfInternationalTrip": False
    }
  ],
  "offerSearchCriteria": {
    "currency": "CHF"
  },
  "anonymousPassengerSpecifications": [
    {
      "externalRef": "123456789",
      "type": "PERSON"
    }
  ],
  "embed": [
    "ALL"
  ]
}

headers = {
  'Authorization': 'Bearer {{agent.token}}',
  'Content-Type': 'application/json'
}

headers['Authorization'] = headers['Authorization'].replace("{{agent.token}}", token)

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

In [106]:
json.loads(response.text)

{'anonymousPassengerSpecifications': [{'externalRef': '123456789',
   'type': 'PERSON'}],
 'trips': [{'destination': {'objectType': 'StopPlaceRef',
    'name': 'Zürich HB',
    'stopPlaceRef': 'urn:uic:stn:8503000'},
   'direction': 'OUT_BOUND',
   'duration': 'PT24H',
   'endTime': '2022-05-16T00:00:00Z',
   'externalRef': 'string',
   'id': 'hTqkqsbZWer46f4N9GKKK0a2MG8o_Qaklla1SQ9Q8lxsYygE-vxC_HMY5xf4hsPLS19k9gG1hNGwNvU3MSs_Vq1BDRzTuRTpPJv3p3tIEdyAuyTLu12TmnUdlUcRoqmUTKS4tzUn3xUdUJk9nVOE4OYJw1o8YwitkJFaWc1xzvqYTG7QGI1UyJiAk2CmpArf6hDenKFkS-VqcuCMFYUcKRYEDz5KPXS23hAkkVi8JBsR54_fhVJ_cXF9dpGWsxHb-qehAtOLf8ggQhrQQy6DinCy2rHoZ0Z2WXgmPUEpahZdRDUZiKu8SRNX4ypUG8jXrSyqVASPy1hVF07amgPKa963IqZt3OpOA59nRSczSXKV3uhLu2heHARYibyfHkWCiuLOuU7KihVybTZqVnyJPIQbgBQ0It4719TdT6LLk_b5C9L6CtZSKU57fY9F7XDtDFWZKw0zsBGiocxODueG7P1jwNQ06o17R5S6CQFkz5UZUhGCXwYMG9yeLFxPMlXLkazHF9g8kG3ReysaEc_dhVDaA9ThtMZIreFMMB__HnQM2Wk6b0jbbMKZ_mhotTJQzflcD7C8VDlB_KuRBJl1LLBvken983oJkzc8dN4b3DsSPya2poi8LAJezUWpyl6LWBLMIVc-',
   

In [86]:
payload

{'tripSearchCriteria': {'origin': 'urn:uic:stn:8500010',
  'destination': 'urn:uic:stn:8503000',
  'departureTime': '2022-05-15T00:00:00+00:00'},
 'passengerSpecification': [{'type': 'PERSON', 'dateOfBirth': '1970-01-01'}],
 'offerSearchCriteria': {'currency': 'CHF'}}